**Objective:** Document my thought process as I develop the function for '/v1/country-data/' endpoint

In [10]:
import pandas as pd
import os


# Read the data

In [13]:
df_assessments = pd.read_excel(r"C:\Users\kbhatia2\Desktop\DS205\ascor-api\data\TPI ASCOR data - 13012025\ASCOR_assessments_results.xlsx")
df_assessments['Assessment date'] = pd.to_datetime(df_assessments['Assessment date'])
df_assessments['Publication date'] = pd.to_datetime(df_assessments['Publication date'])


C:\Users\kbhatia2\AppData\Local\Temp\ipykernel_7944\1610718776.py:2: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df_assessments['Assessment date'] = pd.to_datetime(df_assessments['Assessment date'])


# The function I am designing

In [25]:
def get_country_data(country: str, assessment_year: int):

    selected_row = (
        (df_assessments["Country"] == country) &
        (df_assessments['Assessment date'].dt.year == assessment_year)
    )

    # Filter the data
    data = df_assessments[selected_row]

    if data.empty:
        raise ValueError("There are no data points for the combination of: {country}, {assessment_year}")

    # Selected and filter columns
    area_columns = [col for col in df_assessments.columns if col.startswith("area")]
    data = data[area_columns]
    
    # JSON does not allow for NaN or NULL. 
    # The equivalent is just to leave an empty string instead
    data = data.fillna('')

    #Rename columns
    data['country'] = country
    data['assessment_year'] = assessment_year

    remap_area_column_names = {
        col: col.replace('area ', '')
        for col in area_columns
    }

    data = data.rename(columns=remap_area_column_names)

    # Grab just the first element (there should only be one anyway)
    # and return it as a dictionary
    output_dict = data.iloc[0].to_dict()

    return output_dict

Test how the function behaves

In [27]:
get_country_data('Italy', 2024)

{'EP.1': 'Partial',
 'EP.2': 'Partial',
 'EP.3': 'Partial',
 'CP.1': 'No',
 'CP.2': 'Partial',
 'CP.3': 'Partial',
 'CP.4': 'Partial',
 'CP.5': 'Partial',
 'CP.6': 'Partial',
 'CF.1': 'No',
 'CF.2': 'Exempt',
 'CF.3': 'Partial',
 'CF.4': '',
 'country': 'Italy',
 'assessment_year': 2024}